In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# The List of 20 type of cryptocurrency
coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'LINK', 'USDC', 'BCH', 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']
#coin_list = ['BTC']

In [22]:
#defining the dataframe
main_df = pd.DataFrame()

In [23]:
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])
    # Defining the Start Date and End Date
    datetime_start = datetime(2022, 7, 1, 0, 0)
    datetime_end = datetime(2022, 7, 2, 0, 0)
    step = datetime_start + relativedelta(hours= 12)
    dates = [[datetime_start, step],[step, datetime_end]]

    for date in dates:
        #Api for the scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+date[0].strftime("%Y-%m-%dT%H:%M") + '&end_date=' + date[1].strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'
        print(url)
        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        #print(temp_data)
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        print(df)
        
        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data. 
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will 
        be substituted for the missing one.
        '''
        
        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [date[1] - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
        coin_df = df.append(coin_df)
        
    coin_df['Symbol'] = coin
    main_df = main_df.append(coin_df)


https://production.api.coindesk.com/v2/price/values/BTC?start_date=2022-07-01T00:00&end_date=2022-07-01T12:00&ohlc=true
         Timestamp          Open          High           Low         Close
0    1656633659000  19908.548167  19928.646708  19821.358143  19826.727036
1    1656633719000  19823.212510  19834.419008  19782.602360  19815.995686
2    1656633899000  19776.460487  19776.460487  19739.044738  19761.885168
3    1656633959000  19760.627701  19765.984674  19687.094002  19705.513131
4    1656634019000  19707.473504  19774.229139  19698.521267  19740.733121
..             ...           ...           ...           ...           ...
704  1656676559000  19142.990952  19162.429388  19139.156405  19162.310949
705  1656676619000  19162.370526  19186.315568  19158.176725  19177.577184
706  1656676679000  19177.268689  19200.099436  19157.170412  19159.835866
707  1656676739000  19159.868732  19177.284241  19159.756572  19177.284241
708  1656676799000  19177.284235  19181.197167  19171.5

In [24]:
main_df.head()

,Open,High,Low,Close,Datetime,Symbol
0,19174.670226,19179.136371,19169.209550,19176.898691,2022-07-01 13:07:00,BTC
1,19183.297681,19189.366100,19183.297681,19189.218983,2022-07-01 13:08:00,BTC
2,19198.731974,19209.176805,19188.486589,19191.070781,2022-07-01 13:09:00,BTC
3,19187.512601,19187.512690,19175.428724,19179.743906,2022-07-01 13:10:00,BTC
4,19176.979291,19184.066983,19175.718268,19184.066983,2022-07-01 13:11:00,BTC


In [25]:
len(main_df)

25878

In [26]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df

,Datetime,Symbol,Open,High,Low,Close
0,2022-07-01 13:07:00,BTC,19174.670226,19179.136371,19169.209550,19176.898691
1,2022-07-01 13:08:00,BTC,19183.297681,19189.366100,19183.297681,19189.218983
2,2022-07-01 13:09:00,BTC,19198.731974,19209.176805,19188.486589,19191.070781
3,2022-07-01 13:10:00,BTC,19187.512601,19187.512690,19175.428724,19179.743906
4,2022-07-01 13:11:00,BTC,19176.979291,19184.066983,19175.718268,19184.066983
...,...,...,...,...,...,...
25873,2022-07-01 11:55:00,EOS,0.898605,0.899697,0.897748,0.899694
25874,2022-07-01 11:56:00,EOS,0.899695,0.901464,0.899673,0.900575
25875,2022-07-01 11:57:00,EOS,0.900578,0.902397,0.899669,0.899677
25876,2022-07-01 11:58:00,EOS,0.899702,0.900661,0.899672,0.900661


In [27]:
main_df.to_csv('main_df.csv', index=False)

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
path = f'/content/drive/MyDrive/Colab Notebooks/Project/data/main_df.csv'

#/content/drive/MyDrive/Colab Notebooks/Project/data

In [43]:
with open(path, 'w', encoding = 'utf-8-sig') as f:
  main_df.to_csv(f)